In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("customer_churn").getOrCreate()

## Import data

In [69]:
data = spark.read.csv("customer_churn.csv",inferSchema=True,header=True)

In [70]:
data.printSchema()
data.show(5)

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- Churn: integer (nullable = true)

+----------------+----+--------------+---------------+-----+---------+-------------------+--------------------+--------------------+-----+
|           Names| Age|Total_Purchase|Account_Manager|Years|Num_Sites|       Onboard_date|            Location|             Company|Churn|
+----------------+----+--------------+---------------+-----+---------+-------------------+--------------------+--------------------+-----+
|Cameron Williams|42.0|       11066.8|              0| 7.22|      8.0|2013-08-30 07:00:40|10265 Elizabeth M...|          Harvey LLC|    1|
|   Kevin Mueller|41.0|      1

## Add a column 'days_since_Onboard' until now

In [72]:
# add a column 'days_since_Onboard' until now

from pyspark.sql.functions import datediff, current_date
data = data.withColumn("days_since_Onboard",
                       datediff(current_date(),data['Onboard_date']).alias("days_since_Onboard"))

data.printSchema()
data.show(5)

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- Churn: integer (nullable = true)
 |-- days_since_Onboard: integer (nullable = true)

+----------------+----+--------------+---------------+-----+---------+-------------------+--------------------+--------------------+-----+------------------+
|           Names| Age|Total_Purchase|Account_Manager|Years|Num_Sites|       Onboard_date|            Location|             Company|Churn|days_since_Onboard|
+----------------+----+--------------+---------------+-----+---------+-------------------+--------------------+--------------------+-----+------------------+
|Cameron Williams|42.0|       11066.8|              0| 7.22| 

## Select only columns with numeric data types

In [73]:
cols = []
for dataPoint in data.dtypes:
    if ((dataPoint[1]=='double') or (dataPoint[1]=='int')):
        cols.append(dataPoint[0]) 

In [137]:
data_model = data.select(cols)
data_model.show(5)

+----+--------------+---------------+-----+---------+-----+------------------+
| Age|Total_Purchase|Account_Manager|Years|Num_Sites|Churn|days_since_Onboard|
+----+--------------+---------------+-----+---------+-----+------------------+
|42.0|       11066.8|              0| 7.22|      8.0|    1|              1553|
|41.0|      11916.22|              0|  6.5|     11.0|    1|              1570|
|38.0|      12884.75|              0| 6.67|     12.0|    1|               519|
|42.0|       8010.76|              0| 6.71|     10.0|    1|              1318|
|37.0|       9191.58|              0| 5.56|      9.0|    1|               681|
+----+--------------+---------------+-----+---------+-----+------------------+
only showing top 5 rows



In [138]:
print('Total customers:', data_model.count())
print('Customers with no account manager:', data_model.filter(data_model['Account_Manager']==0).count())
print('Customers with account manager:', data_model.filter(data_model['Account_Manager']==1).count())

Total customers: 900
Customers with no account manager: 467
Customers with account manager: 433


## The goal is to predict whether a customer will churn without an account manager assigned
## Since account manager is randomly assigned, let's not put it in the model and select only customer with no account manager

In [141]:
data_model = data_model.filter(data_model['Account_Manager']==0)
print('Customer with no account manager to use in the model:', data_model.count())

data_model.show(5)

Customer with no account manager to use in the model: 467
+----+--------------+---------------+-----+---------+-----+------------------+
| Age|Total_Purchase|Account_Manager|Years|Num_Sites|Churn|days_since_Onboard|
+----+--------------+---------------+-----+---------+-----+------------------+
|42.0|       11066.8|              0| 7.22|      8.0|    1|              1553|
|41.0|      11916.22|              0|  6.5|     11.0|    1|              1570|
|38.0|      12884.75|              0| 6.67|     12.0|    1|               519|
|42.0|       8010.76|              0| 6.71|     10.0|    1|              1318|
|37.0|       9191.58|              0| 5.56|      9.0|    1|               681|
+----+--------------+---------------+-----+---------+-----+------------------+
only showing top 5 rows



In [142]:
data_model = data_model.drop('Account_Manager')
data_model.show(5)

+----+--------------+-----+---------+-----+------------------+
| Age|Total_Purchase|Years|Num_Sites|Churn|days_since_Onboard|
+----+--------------+-----+---------+-----+------------------+
|42.0|       11066.8| 7.22|      8.0|    1|              1553|
|41.0|      11916.22|  6.5|     11.0|    1|              1570|
|38.0|      12884.75| 6.67|     12.0|    1|               519|
|42.0|       8010.76| 6.71|     10.0|    1|              1318|
|37.0|       9191.58| 5.56|      9.0|    1|               681|
+----+--------------+-----+---------+-----+------------------+
only showing top 5 rows



In [143]:
vecCols = data_model.columns
vecCols.remove('Churn')
vecCols

['Age', 'Total_Purchase', 'Years', 'Num_Sites', 'days_since_Onboard']

In [144]:
from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler(inputCols=vecCols,outputCol='features')
data_feed = assembler.transform(data_model).select('Churn','features')
data_feed.show(5)

+-----+--------------------+
|Churn|            features|
+-----+--------------------+
|    1|[42.0,11066.8,7.2...|
|    1|[41.0,11916.22,6....|
|    1|[38.0,12884.75,6....|
|    1|[42.0,8010.76,6.7...|
|    1|[37.0,9191.58,5.5...|
+-----+--------------------+
only showing top 5 rows



In [145]:
train_data, test_data = data_feed.randomSplit([0.7, 0.3])

## Build a model

In [146]:
from pyspark.ml.classification import LogisticRegression

logReg = LogisticRegression(featuresCol='features',labelCol='Churn')
logReg_trained = logReg.fit(train_data)

In [150]:
logReg_trained.summary.predictions.show(5)

+-----+--------------------+--------------------+--------------------+----------+
|Churn|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|[26.0,8939.61,4.5...|[8.24435044556657...|[0.99973733028454...|       0.0|
|  0.0|[28.0,8670.98,3.9...|[9.56471671670916...|[0.99992984383522...|       0.0|
|  0.0|[28.0,11204.23,3....|[3.17279429372938...|[0.95979754444224...|       0.0|
|  0.0|[29.0,9378.24,4.9...|[5.56045202975208...|[0.99616770594780...|       0.0|
|  0.0|[29.0,9617.59,5.4...|[5.47788020783679...|[0.99583920734451...|       0.0|
+-----+--------------------+--------------------+--------------------+----------+
only showing top 5 rows

